### 운영체제(OS) 개요

- 배치 시스템 (Batch System)
    - 여러가지 작업을 모아서 순서대로 실행하는 시스템
        
        - 작업이 한 번에 하나씩 실행 되어, CPU를 100% 활용할 수 없음
        
        - 사용자가 즉각 피드백을 받을 수 없다.
        
        - ex) 코드를 넣고 돌린 후 1일 뒤에 이 코드의 결과값을 알아볼 수 있다.... (오타라도 있었다면? 끔찎)

- 프로세스란? 
    - 프로그램(exe파일 등)은 하드디스크에 저장되어있는 어떠한 조각들의 모음집이다.

    - 그 프로그램의 인스턴스 하나를 프로세스라고 부른다.

    - 그렇기 때문에 같은 프로그램이라도, 여러번 실행한다면 여러개의 프로세스가 생성되며, 그 들은 서로 독립적이다.



- 멀티태스킹 (Multitasking)
    - 여러 프로세스를 CPU가 빠르게 전환(Switching) 하면서 실행하여, 마치 동시에 실행되는 것 같은 느낌을 준다.

    - Context Switching
        - 프로세스의 실행 상태를 나타내는 정보 (state)
            - 레지스터, PC, SP 등등... 쉽게 말해 프로세스간 전환이 필요할 때, 멈춘 상태부터 다시시작하기 위한 정보를 저장하고 로드한다는 개념
        
        - 컨텍스트 스위칭이 많아지면, 오버헤드 발생 , CPU 리소스를 낭비한다. 그러므로, 최소화 해야한다.

    - 디스크 I/O가 많아짐에 따른 성능 저하
        - 메모리가 부족할 때, 하드디스크의 일부를 메모리처럼 사용하는 가상메모리 상황에서, 스와핑이 많이 발생하게 되면서 속도가 느려지는 상황이 발생한다.
        - RAM의 크기가 충분하다면 문제가 크지 않게 된다.
        
    - 여러 프로세스가 자원을 공유하게 되면서, 병렬 처리 시 동기화 문제(데드락, 레이스 컨디션 등) 가 발생한다.
        - 그러므로 프로세스 간 경쟁이 발생 -> 동기화 및 스케쥴링이 필요하다.

- 시분할(Time-Sharing) 스케쥴링
    - CPU 시간을 잘게 쪼개서 프로세스들에게 배분하여, 동시에 처리되는 듯한 효과(일루전)를 제공한다.

    - 멀티태스킹과 다르게 고정된 시간할당(타임 퀀텀)이기 때문에, 시간이 지나면 강제로 cpu를 교체하므로, 스케줄링이 예측하능하여, 과부화 제어가 좀 더 쉽다.
    
    - CPU 할당 시간을 조절하여 효율적으로 메모리를 활용한다.
    
    - CPU가 사용자모드 <-> 커널 모드 전환을 반복하며 오버헤드가 증가한다.

    - 잘못된 스케줄링(ex: 너무 짧은 타임슬라이스)은 성능저하를 초래할 수 있다.

    - 실시간 시스템에는 적합하지 않다.

#### FORK , BLOCK , SUSPENDED , SWAPPING , Interrupt & Exception

- fork : 하나의 코드, 두 개의 실행 흐름
    - fork()는 부모 프로세스가 호출 시 **새로운 자식 프로세스(child)**를 생성합니다.
    - fork()는 멀티 프로세스 생성에 사용되며, 병렬 처리가 필요할 때 유용합니다.
    - 부모와 자식 프로세스는 동일한 코드와 데이터를 공유하지만, 각각 독립적인 실행 흐름을 가진다.
    
    - fork의 반환값:
        - 부모 프로세스: 자식의 PID를 반환
        - 자식 프로세스: 0을 반환
        - 실패 시: -1 반환
    - 예시:
        - 파워포인트 작업 중 프린트 I/O 요청 시,
        - 기존 방식: I/O 작업 때문에 파워포인트가 멈춤 (Block)
        - fork() 사용: 자식 프로세스가 프린트 작업 처리, 부모 프로세스는 파워포인트 작업 계속


- Block (Blocked 상태) : 대기 중인 프로세스

    - 정의: 프로세스가 I/O 작업 등으로 인해 실행할 수 없을 때 Block 상태가 됩니다.
    
    - 상황:
        - 실행 중(Running)이던 프로세스가 디스크 I/O 요청 시 Block 상태로 전환
        - I/O 작업 완료 후 Ready Queue로 이동하여 CPU 할당을 기다림
        - Block 상태에서는 CPU를 사용하지 않는다.

- Suspended (중단 상태) : 메모리 부족 시 발생
    - 정의: 프로세스가 메모리 공간 확보를 위해 일시적으로 실행이 중단된 상태입니다.
    - 특징:
        - 메모리 부족 시 OS가 프로세스를 강제로 suspend
        - Block 상태와의 차이: Block은 I/O 대기, Suspend는 메모리 문제
    - 용도:
        - 낮은 우선순위 프로세스를 중단하여 다른 프로세스에 메모리 자원 할당
        - Suspend 된 프로세스는 **디스크에 저장(swapping)**되어 메모리를 비웁니다.

- Swapping (스와핑) : 메모리 관리 기법
    - 정의: 실행 중인 프로세스를 메모리에서 디스크로 이동시키는 작업입니다.
    - 목적:
        - DRAM 공간 확보
        - 자주 사용하지 않는 프로세스를 디스크에 저장
    - 과정:
        - 자주 사용하지 않는 프로세스를 디스크로 이동
        - 필요한 프로세스를 메모리로 로드
    - 단점: 디스크 접근 속도가 느려 전체 성능 저하 가능

- Interrupt (인터럽트) : 비동기적, 외부 이벤트 발생
    - 정의: CPU 외부에서 발생하는 예상치 못한 이벤트로, 현재 실행 중인 작업을 중단하고 처리합니다.
    - 특징:
        - 비동기적: 언제 발생할지 예측 불가
    - 예시:
        - 키보드 입력
        - 마우스 클릭
        - 실수로 재부팅 버튼 클릭

- Exception (예외) : 동기적, 내부 이벤트 발생
    - 정의: 프로그램 실행 중 발생하는 내부적인 오류나 예외 상황입니다.
    - 특징:
        - 동기적: 항상 동일한 코드 위치에서 발생
        - 발생 시 운영체제의 핸들러 호출
    - 예시:
        - 0으로 나누기 연산
        - 잘못된 메모리 접근
        - 오버플로우

- 인터럽트 / 익셉션 발생시 - > supended , 하드웨어에 기록된 핸들러 실행 후 프로그램 카운터에 기록된 시점부터 다시 resume
- Exceptions 
    - Traps : 의도적인 예외, 그 다음부터 명령어 실행 (디버깅 등)
    - Faults : 의도하지 않았지만, 복구가 가능 , resume 시 그 시점 부터 실행(current)
    - Aborts : 심각한 하드웨어적 에러 (parity error), 심각하면 시스템을 shut-down
- Interrupt
    - CPU pins : #INT , #NMI
        - CPU에 있는 인터럽트 핀에 assert가 들어오면 인식 후 처리

- UNIX System
    - demon : ftp server 등등.. (백그라운드 커널프로세스), 시작부터가 OS code
    - User Process : 사용자 프로세스와 System Process(커널모드) 왔다갔다하면서 실행


### 스레드(Thread)
- 스레드란?
    - 프로세스 내에서 실행되는 작업의 단위
    - CPU가 실제로 작업을 수행하는 최소 단위
    - 하나의 프로세스 안에서 여러 스레드를 만들어 동시에 여러 작업을 처리할 수 있어요.

- 멀티스레드 사용 예시
    - 크롬 브라우저: 각 탭이 개별 스레드처럼 동작 → 한 탭이 멈춰도 다른 탭은 정상 작동
    - 워드 프로세서:
        - 한 스레드: 화면 렌더링
        - 다른 스레드: 사용자 입력 처리
        - 또 다른 스레드: 오토세이브
    - 웹 서버:
        - 각 클라이언트 요청을 개별 스레드로 처리 → 서버 응답 속도 향상

- 멀티스레딩 특징
    - 같은 프로세스 내에서 메모리를 공유
    - 코드, 데이터, 힙 영역은 공유
    - 스택 영역은 각 스레드마다 따로 있음 (함수 호출, 지역 변수 관리)
    - 빠른 컨텍스트 스위칭:
        - 프로세스 전환보다 스레드 전환이 훨씬 빠름

- 왜 스레드를 사용할까?
    - 빠른 속도: 프로세스보다 훨씬 가볍고 전환 비용이 낮음
    - 자원 절약: 메모리를 공유하므로 효율적
    - 응답성 향상: 사용자 인터페이스가 멈추지 않게 처리 가능

- 프로세스(Process)
    - 프로세스란?
        - 실행 중인 프로그램
        - 운영체제에서 자원을 할당받아 실행되는 독립적인 작업 단위
        - 각 프로세스는 독립된 메모리 공간(코드, 데이터, 힙, 스택)을 가짐
    - 멀티프로세스 특징
        - 서로 메모리 공간을 공유하지 않음
        - 각 프로세스는 보호된 독립 영역을 가짐
        - 프로세스 간 통신(IPC) 필요 (파이프, 소켓, 공유 메모리 등)
        - 안정적이지만 컨텍스트 스위칭 비용이 큼

    - 왜 프로세스를 사용할까?
        - 안정성: 하나의 프로세스가 죽어도 다른 프로세스에는 영향 없음
        - 보안성: 메모리 공유가 없어 안전함


### User-Level Threads (ULT)

#### 장점
- **가볍고 빠름:** 생성 및 관리가 커널의 개입 없이 사용자 영역에서 처리됩니다.
- **낮은 오버헤드:** 커널 모드 전환이 필요 없어 컨텍스트 스위칭 비용이 적습니다.

#### 단점
- **블로킹 문제:** 하나의 스레드가 시스템 콜(I/O)로 블록되면 전체 프로세스가 블록됩니다.
- **병렬 처리 불가:** 커널은 프로세스 단위로만 스케줄링하므로 멀티코어를 사용할 수 없습니다.
- **커널 미인식:** 스케줄링이 사용자 영역에서 처리되어 커널이 이를 인식하지 못합니다.

---

### Kernel-Level Threads (KLT)

#### 장점
- **병렬 처리 가능:** 커널이 스레드를 인식하여 멀티코어에서 병렬 실행할 수 있습니다.
- **독립적 실행:** 하나의 스레드가 블록되어도 다른 스레드는 정상 실행됩니다.
- **커널 스케줄링:** 커널이 직접 스레드를 관리하므로 최적화된 스케줄링이 가능합니다.

#### 단점
- **컨텍스트 스위칭 오버헤드:** 커널 모드 전환으로 비용이 발생합니다.
- **ULT 대비 무거움:** 약 10배 정도의 오버헤드 차이가 있습니다.
- **높은 리소스 소비:** 스레드 생성 및 관리 시 커널 리소스를 더 많이 사용합니다.

---

### Combined Approach (Hybrid Model)

#### 목적
- **ULT와 KLT의 장점 결합:** 성능과 유연성을 동시에 확보합니다.
- **효율적인 스레드 매핑:** 사용자 영역에서 스레드를 관리하되, 커널 스레드와 매핑하여 병렬 처리를 지원합니다.

#### 구현 방식
- **Many-to-One:** 여러 유저 스레드가 하나의 커널 스레드에 매핑됩니다.
- **Many-to-Many:** 여러 유저 스레드가 여러 커널 스레드에 매핑됩니다.
- **대표 예시:** Solaris, Windows NT, Linux NPTL (Native POSIX Thread Library).

---

### 암달의 법칙 (Amdahl's Law)

#### 개념
- **병렬 처리의 최대 속도 향상은 직렬 부분에 의해 제한됩니다.**
- 멀티코어 사용 시 프로세스 간 통신과 작업 분배로 인해 오버헤드가 발생합니다.

#### 수식
\[
$S = \frac{1}{(1 - P) + \frac{P}{N}}$
\]
- **S:** 이론적 최대 속도 향상 비율  
- **P:** 병렬화 가능한 작업 비율  
- **N:** 사용한 프로세서 수  

#### 예시
- 작업 중 80%가 병렬화 가능하고 4코어 사용 시 속도 향상은 \( $S = \frac{1}{(1 - 0.8) + \frac{0.8}{4}} = 2.5$ \)배입니다.

---

### 스레드 상태 전이 과정

1. **생성 (Create):** 스레드 생성.
2. **레디 (Ready):** 실행 대기 상태.
3. **실행 (Run):** CPU가 할당되어 실행 중.
4. **블록 (Block):** I/O 요청 등으로 대기 상태.
5. **서스펜드 (Suspend):** 메모리 부족 시 디스크로 스와핑.
6. **좀비 (Zombie):** 실행 완료 후 부모 프로세스가 아직 회수하지 않음.
7. **종료 (Terminate):** 자원이 회수되어 완전 종료.

---

### Process Interaction (프로세스 상호작용)

#### Mutual Exclusion (상호 배제)
- **목적:** 공유 자원의 동시 접근 방지.
- **해결책:** 뮤텍스(Mutex), 세마포어(Semaphore), 모니터(Monitor) 사용.

#### Deadlock (교착 상태)
- **정의:** 여러 프로세스가 서로 자원을 기다리며 무한 대기 상태에 빠짐.
- **예방 기법:**
  - 자원 할당 순서 지정
  - 교착 상태 탐지 및 회복
  - 자원 강제 회수

#### Starvation (기아 상태)
- **정의:** 낮은 우선순위 작업이 자원을 영원히 할당받지 못하는 현상.
- **해결책:**
  - 우선순위 상향 조정
  - 공정한 스케줄링(FIFO, Round Robin) 적용

#### Race Condition (경쟁 상태)
- **정의:** 두 개 이상의 프로세스가 동시에 자원에 접근할 때 발생.
- **해결책:**
  - 상호 배제 기법 적용
  - 원자적 연산 보장 (Atomic Operation)

---

### 요약
| 구분 | 장점 | 단점 |
|---|---|---|
| **ULT** | 빠르고 오버헤드 낮음 | 블로킹 문제, 병렬 처리 불가 |
| **KLT** | 병렬 처리 지원, 독립 실행 가능 | 높은 컨텍스트 스위칭 오버헤드 |
| **Hybrid** | 두 접근법의 장점 결합 | 구현 복잡성 증가 |

- **암달의 법칙:** 병렬 처리의 이론적 한계를 설명.
- **프로세스 상호작용:** 동시성 문제 예방 필수.

---


## Atomic / Semaphore / Mutex / Spinlock

- **Atomic** (vs Mutex : 연산 단위)
    - 한 번에 끝나는 연산 (원자적 연산)
    - 연산 도중 끼어들거나 중단되지 않는다.
    - 아주 짧은 작업(단일 변수)에 대해 race condition을 원천 차단.
    - 하지만 긴 코드 블록은 보호 못한다.

- **Semaphore** (vs Mutex : 소유권 X, vs Semaphore : 카운팅만 함)
    - 크리티컬 섹션에 동시 접근 가능한 자원 개수를 제한, 관리
    - 세마포어 값이 0이면 접근 불가, 양수면 접근 가능
    - P(): Wait(자원 얻기) → 세마포어 값 감소
    - V(): Signal(자원 반환) → 세마포어 값 증가
    - 소유권이 없어서 wait한 놈과 signal하는 놈이 달라도 된다.
    - ex: 프린터 2대 → 2명까지만 동시에 사용 가능

- **Mutex** (vs Semaphore : 소유권 O, vs Atomic : 블록 단위 보호)
    - 한 번에 하나만 접근 가능하도록 강제하는 Lock
    - Locked / Unlocked 두 가지 상태
    - 락을 건 스레드만 해제할 수 있음 (소유권 있음)
    - 긴 크리티컬 섹션 보호 + 스케줄러 선점 제어

- **Spinlock** (vs Mutex : CPU 바쁨/쉬기 차이)
    - atomic instruction을 이용해 레지스터와 메모리 값을 비교/교환하며 lock 획득 시도
    - lock이 풀릴 때까지 루프를 돌면서 busy-wait
    - CPU를 쉬지 않고 계속 굴림 → 짧은 작업에 적합
    - 긴 작업엔 쓰면 안 됨 (CPU 리소스 낭비 심각)


### 모니터와 조건 변수

#### 모니터란?
- **뮤텍스 + 조건 변수** 를 합친 고수준 동기화 도구
- 임계 구역 보호 + 조건 충족 전까지 대기까지 관리함
- 조건 안 되면 wait, 조건 되면 notify로 깨어남

---

#### 뮤텍스 vs 모니터

| | 뮤텍스 | 모니터 |
|--|--|--|
| 보호 범위 | 임계 구역만 | 임계 구역 + 조건 대기 |
| 조건 대기 | 불가능 | 가능 |
| 스레드 관리 | 단순 락/언락 | 락 + 대기 + 깨우기 관리 |

---

#### 조건 변수(cv) + 모니터 설계

- 조건 안 되면 → `wait()` → 자동으로 unlock + block → CPU 절약
- 조건 되면 → `notify` → 대기 중인 스레드 깨움
- 깨어난 스레드는 → 다시 lock 잡고 → **while 조건 확인 후 작업**
- 깨어났지만 조건 불충족 → 다시 잠

---

#### notify_one vs notify_all

| | notify_one | notify_all |
|--|--|--|
| 깨우는 대상 | 아무나 1명 (보장 X) | 모두 |
| 효율성 | 높음 (과부하 적음) | 낮음 (모두 깨움 → 과부하 가능성) |
| 사용 시기 | 조건 충족 가능성 높을 때 | 어떤 스레드가 조건 만족할지 모를 때 |

---

#### unique_lock vs lock_guard

- 뮤텍스 소유 및 관리 도구 (락/언락 자유로움)
- `wait()` 같은 동작 위해서 **unique_lock 필수**
- `lock_guard`는 스코프 끝에서만 unlock → 조건변수와 호환 불가

```cpp
std::unique_lock<std::mutex> lock(mtx);
cv.wait(lock); // 내부에서 unlock → block → 깨어나면 다시 lock
```
